# Forecast examples
This notebook demonstrates how to read and upload forecasts

In [ ]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

In [ ]:
import pandas as pd
import numpy as np
from hps_api_client.apiproxy import ApiProxy

## Create a API client and query forecasts

In [ ]:
client = ApiProxy(uri = "http://leviathan:5400/api/v1/")

Forecasts are tied to a hydro system and we will start by listing those available and choosing Flørli as the system we will be working on

In [ ]:
hydro_systems = client.get_hydro_systems()
selected_hydro_system = hydro_systems[0]
selected_hydro_system

We can now list all forecasts available for our hydro system.

In [ ]:
forecasts = client.get_forecasts(selected_hydro_system)
forecasts

Lets pick one and list available scenarios.

In [ ]:
selected_forecast = forecasts[0]
scenarios = client.get_forecast_scenarios(selected_forecast)
print(scenarios)

We can retrieve data for a single scenario...

In [ ]:
client.get_scenario_data(selected_forecast, "2014")

.. or read all scenarions in one go.

In [ ]:
data = client.get_forecast_data(selected_forecast)
data

Note that the dataframe above has a MultiIndex and is accessed by both scenario and series type.

In [ ]:
data[("Price", "2013")].plot()

Alternativly we can access all prices or inflows by using the primary index.

In [ ]:
data["Price"].plot(figsize=(20,13))

We now want to add a new forecast to our chosen hydro system. The add_forecast method accepts a pandas DataFrame of the same format that we retrieved above. Lets build it..

In [ ]:
data = {}
dates = pd.date_range("2020-12-31 23:00:00+00:00", "2025-12-31 23:00:00+00:00",freq='1d')

for year in range(2000, 2030):
    scenario = str(year)
    prices = np.random.random(len(dates)) * 60
    inflows = np.random.random(len(dates)) * 20
    
    data[("Price", scenario)] = prices
    data[("Inflow", scenario)] = inflows
    
df = pd.DataFrame(data)
df["Date"] = dates
df.set_index("Date", inplace=True)

df

## Upload a forecast
When uploading we have to specify a name for the new forecast and the hydro system that we want to associate it with.

In [ ]:
client.add_forecast("Demo forecast", selected_hydro_system, df)

Finally, lets verify that the forecast is in place and ready for training and evaluation.

In [ ]:
new_forecast = client.get_forecasts(selected_hydro_system)[-1]
client.get_forecast_data(new_forecast)